In [ ]:
import pandas as pd

from src.all_in_one import *

In [ ]:
# Screening
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr = get_df(scr_file.path, scr_file.sheet)
unique_scr = scr.drop_duplicates(subset='rid')

In [ ]:
td_file = path_manager.get_data_file(Category.PS, PSFile.TD)
td_dataset = Dataset(config_file, td_file.sheet)
td = get_df(td_file.path, td_file.sheet)
td = pd.merge(td, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# filtered_td = filter_dataframe_on_date(td, td_dataset.dvars, start, end, FilterType.IN)

In [ ]:
td

In [ ]:
# filtered_td

In [ ]:
# filtered_td.to_clipboard(index=False)

In [ ]:
tdid = pd.read_excel("../../tdid.xlsx")
# import os
# print(os.getcwd())
# print(os.listdir(".."))

In [ ]:
tdid

In [ ]:
cws_dataset = Dataset(config_file, cws_sheet)
cws = get_df(cws_path, cws_sheet)

In [ ]:
# cws

In [ ]:
# td_cws = pd.merge(td, cws[['rid', 'cws_s1', 'cws_s2']], on='rid', how='left')

In [ ]:
td_cws = pd.merge(tdid, cws[['rid', 'cws_s1', 'cws_s2']], on='rid', how='left')

In [ ]:
# td_cws

In [ ]:
adid = pd.read_excel("../../adid.xlsx")

In [ ]:
adid = adid.sort_values(by='ad_date')
adid = adid.drop_duplicates(subset='rid')

In [ ]:
adid

In [ ]:
td_adv = pd.merge(tdid, adid[['rid', 'ad_date']], on='rid', how='left')

In [ ]:
td_adv

In [ ]:
td_adv[td_adv['rid'].duplicated()]['rid'].unique()